**Context:** You are a data scientist at Santander and have been tasked with engaging your customers in a more personalized way. Your goal is to use the power of Generative AI to create personalized marketing messages that will be delivered to each customer.

**Problem Conditions:**

1. You received a simple spreadsheet, in CSV format ('SDW2023.csv'), with a list of bank user IDs:
   ```
   UserID
   1
   two
   3
   4
   5
   ```
2. Your job is to consume the `GET https://sdw-2023-prd.up.railway.app/users/{id}` endpoint (Santander Dev Week 2023 API) to get the data of each customer.
3. After getting customer data, you will use the ChatGPT API (OpenAI) to generate a personalized marketing message for each customer. This message should emphasize the importance of investments.
4. Once the message to each customer is ready, you'll send that information back to the API, updating each user's "news" list using the `PUT https://sdw-2023-prd.up' endpoint .railway.app/users/{id}`.

# Extract
Extracting list of user IDs from CSV file. For each ID, make a GET request to get the corresponding user data.

In [1]:
import pandas as pd

df = pd.read_csv("usuarios.csv")
user_ids = df["UserID"].tolist()
print(user_ids)


[77, 78, 79]


Consuming an API from the DIO repository

In [2]:
import requests
import json


api_url = "https://sdw-2023-prd.up.railway.app"


def get_user(id):
    response = requests.get(f'{api_url}/users/{id}')
    return response.json() if response.status_code == 200 else None

In [3]:
# Percorre a lista de user_ids e para cada ida vai fazer uma chamada na api
users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(user, indent=2))

{
  "id": 79,
  "name": "Carla",
  "account": {
    "id": 86,
    "number": "0113-1",
    "agency": "2039",
    "balance": 0.0,
    "limit": 500.0
  },
  "card": {
    "id": 79,
    "number": "**** **** **** 2511",
    "limit": 1000.0
  },
  "features": [],
  "news": []
}


# Transform
Generating a personalized message for each user using the OpenAI GPT-3 API

In [4]:
#instalando 
#!pip install openai

In [5]:
openai_api_key = "insira aqui a sua chave"

In [6]:
import openai

openai.api_key = openai_api_key

In [7]:
# Função para gerar a mensagem para os usuários
def generate_ai_news(user):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
        {
            "role": "system", "content": "You are a banking marketing expert."
        },
        {
            "role": "user", "content": f"create a message for {user['name']} about the importance of investments (maximum 100 characters)"
        }
    ]
    )

    return completion.choices[0].message.content.strip('\"')


In [ ]:
for user in users:
    news = generate_ai_news(user)
    print(news)
    user['news'].append({
        "description" : news
    })

# LOAD
Update each user's ta with the new generated message.

In [ ]:
def update_user(user):
    response = requests.put(f"{api_url}/users/{user['id']}", json=user)
    return True if response.status_code == 200 else False

for user in users:
    success = update_user(user)
    print(f"User {user['name']} updated? {success}!")